In [1]:
from source.regency import Regency
from source.run import rungame
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
%matplotlib notebook

In [3]:
X = 5
my_game = rungame('TestGame', train=True, GameLength=X, IntDC = 15)
Game = my_game.Game


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Continuing Game TestGame at Season 0 Action 3


In [ ]:
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs(Update=True)
Game.set_override('WM', 45)
Game.take_domain_actions()
Game.take_domain_actions()
Game.take_domain_actions()
Game.War

In [6]:
print(Game.War['Notes'].values[-1])

Willaim Moergen's forces occupied Sunken Lands, sacking and looting the provinces.


In [7]:
import numpy as np
import pandas as pd
self = Game

Regent, decision, actor, Type, state, capital, high_pop, low_pop, friend, enemy, rando, enemy_capital = Game.errors[2]

IndexError: list index out of range

In [ ]:
np.argmax(decision)